# Fonte de Alimentação Estabilizada

[Ver mais](https://www.notion.so/guilhermebaos/18-Circuitos-com-D-odos-bec037cdcecb4f36a6c0c4d5c225115c?pvs=4#1d22979f33d44f5b87287e51af381d29) - Teoria!

In [39]:
import csv
import numpy as np

# Parâmetros
VP = 20.747
F = 50

RR = np.array([150, 220, 470, 1000, 1500, 2200, 4700, 10000, 1e06])
CC = np.array([100e-06, 1000e-06])


# Tensão nas placas do condensador caso este descarregue alpha * Tau
def est(alpha: float, ftau: float) -> float:
    global VP
    return VP * np.exp(-alpha/ftau)


# Determinar o tempo que o condensador está a descarregar
N = 100001

aa = np.linspace(0, 1, N)
cc = VP * np.cos(2 * np.pi * aa)

valores = []
for c in CC:
    for r in RR:
        ftau = F * r * c
        ee = est(aa, ftau)
        dd = cc - ee

        # O condensador carrega se cc >= ee
        carga = aa[dd >= 0]

        # O tempo de descarga é o tempo entre os momento em que está a carregar
        descarga = np.max(np.diff(carga))
        valores += [(c, r, VP - est(descarga, ftau))]
        print(c, r, VP - est(descarga, ftau))


titulos = ["C/ F", "R/ Ohm", "V_ripple/ V"]
with open("01-Dados.csv", "w+", encoding="UTF-8", newline="") as ficheiro:
    writer = csv.writer(ficheiro)

    writer.writerow(titulos)
    for item in valores:
        writer.writerow(item)
        

0.0001 150.0 13.011030172197728
0.0001 220.0 10.562039174624818
0.0001 470.0 6.2953985937611225
0.0001 1000.0 3.4068361959000697
0.0001 1500.0 2.385055829156965
0.0001 2200.0 1.6831570186719063
0.0001 4700.0 0.8243381412386306
0.0001 10000.0 0.39770027169525335
0.0001 1000000.0 0.004135759441638243
0.001 150.0 2.385055829156965
0.001 220.0 1.6831570186719063
0.001 470.0 0.8243381412386306
0.001 1000.0 0.39770027169525335
0.001 1500.0 0.2676173314933621
0.001 2200.0 0.18370553706295922
0.001 4700.0 0.08679813397064251
0.001 10000.0 0.04103372658694937
0.001 1000000.0 0.0004145180143986238
